# Refactor bus corridors

In [ ]:
import dask.dataframe as dd
import dask_geopandas
import geopandas as gpd
import pandas as pd

from calitp.tables import tbl
from siuba import *

import C1_prep_for_clipping as prep_clip
import C3_clean_clipped_intersections as clean_clip
from A1_rail_ferry_brt import analysis_date
from B1_bus_corridors import TEST_GCS_FILE_PATH
from shared_utils import utils, geography_utils
from utilities import catalog_filepath

# Input files
ALL_BUS = catalog_filepath("all_bus")

segment_cols = ["calitp_itp_id", "hqta_segment_id"]
stop_cols = ["calitp_itp_id", "stop_id"]

In [ ]:
import C4_create_bus_hqta_types as C4

gdf = clean_clip.process_clipped_intersections()

# Don't need to save the buffered gdf here probably
#utils.geoparquet_gcs_export(one_intersection_per_row, 
#                            f'{TEST_GCS_FILE_PATH}', 'major_bus_stops')

freq_bus_stops_in_intersections = C4.create_major_transit_stop(gdf)

# This exploded geometry is what will be used in spatial join on all stops
# to see which stops fall in this
bus_intersections = C4.explode_bus_intersections(freq_bus_stops_in_intersections)

#all_stops = query_all_stops(analysis_date)
all_stops = dask_geopandas.read_parquet("./data/all_stops.parquet")

major_stop_bus = C4.create_major_stop_bus(all_stops, bus_intersections)
stops_in_hq_corr = C4.create_stops_along_corridors(all_stops)


In [ ]:
major_stop_bus2 = major_stop_bus.compute()
stops_in_hq_corr2 = stops_in_hq_corr.compute()

In [ ]:
stops_in_hq_corr2[stops_in_hq_corr2.calitp_itp_id_primary==182].plot(
    column="stop_id", figsize=(10, 10), cmap="tab10")

In [ ]:
major_stop_bus2[major_stop_bus2.calitp_itp_id_primary==182].plot(
    column="stop_id", figsize=(10, 10), cmap="tab10")

In [ ]:
#https://stackoverflow.com/questions/71688126/groupby-map-partitions-in-dask
trips_by_stop_hour = stop_times.map_partitions(
    lambda df: df.groupby(["calitp_itp_id", "stop_id"], as_index=False)
        .agg({"trip_id": "count"}), meta={"calitp_itp_id": int,
                                          "stop_id": str,
                                          "trip_id": int
                                         }
)